In [1]:
import hvplot.polars  # noqa
import polars as pl

from polars_ts.decomposition.fourier_decomposition import fourier_decomposition
from polars_ts.decomposition.seasonal_decomposition import seasonal_decomposition

df = pl.read_csv("https://datasets-nixtla.s3.amazonaws.com/air-passengers.csv", try_parse_dates=True)

In [3]:
fourier = (
    fourier_decomposition(df, freqs=["month", "quarter"], ts_freq=12, n_fourier_terms=10)
    .rename({"seasonal": "seasonal_fourier"})
    .hvplot.line("ds", ["trend", "seasonal_fourier"], line_dash="dashdot")
)

naive = (
    seasonal_decomposition(df, freq=12)
    .rename({"seasonal": "seasonal_naive"})
    .hvplot.line("ds", ["trend", "seasonal_naive"], line_dash="dotdash")
)

fourier * naive

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [ds]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [ds]   (value)

# Time Series with high complexity

Let us now generate a time series with high complexity to see the differences between Fourier and Naive decomposition.

In [4]:
from datetime import date

import numpy as np
import polars as pl

# Set random seed for reproducibility
np.random.seed(42)


# Function to generate complex seasonality with increased noise
def generate_complex_seasonality(n_samples: int, start_date: str, end_date: str):
    # Generate time index (daily frequency)
    time_index = pl.date_range(start=start_date, end=end_date, interval="1d", eager=True)

    # Generate time series with multiple seasonal components and noise
    time_in_days = np.arange(n_samples)

    # Daily seasonality (e.g., hourly pattern repeated for each day)
    daily_seasonality = np.sin(2 * np.pi * time_in_days / 24)  # Cycle repeats every 24 units (e.g., 24 hours)

    # Weekly seasonality (e.g., day of the week pattern)
    weekly_seasonality = np.sin(2 * np.pi * time_in_days / 7)  # Cycle repeats every 7 days (week)

    # Yearly seasonality (e.g., month of the year pattern)
    yearly_seasonality = np.sin(2 * np.pi * time_in_days / 365)  # Cycle repeats every 365 days (year)

    # Add more noise: Increase variance and add a periodic random noise component
    noise = np.random.normal(0, 10, n_samples)  # Higher variance for stronger noise
    random_noise = np.random.normal(0, 1, n_samples)  # Smaller noise component
    periodic_noise = 1 * np.sin(2 * np.pi * time_in_days / 10)  # Additional periodic noise

    # Combine seasonalities with random and periodic noise (to create complexity)
    target = (
        50
        + 20 * daily_seasonality
        + 10 * weekly_seasonality
        + 5 * yearly_seasonality
        + noise
        + random_noise
        + periodic_noise
    )

    # Create the dataframe in Polars
    df = pl.DataFrame({"ds": time_index, "y": target})

    return df


# Generate complex seasonality data (e.g., for 2 years of daily data)
n_samples = 365 * 2 + 1  # 2 years of daily data
start_date = date(2023, 1, 1)
end_date = date(2024, 12, 31)
df_complex = generate_complex_seasonality(n_samples, start_date, end_date).with_columns(pl.lit(1).alias("unique_id"))

df_complex.hvplot("ds", "y", title="Time Series with Complex Seasonality and High Noise")

:Curve   [ds]   (y)

# Plot against each other to see the different seasonal components

* Fourier grows in complexity as you add terms, may filter out some noise. 
* Naive Seasonal will simply gather the period averages of the detrended series.

In [5]:
fourier = (
    fourier_decomposition(
        df_complex, freqs=["day_of_month", "month", "week", "day_of_year"], ts_freq=365, n_fourier_terms=10
    )
    .rename({"seasonal": "seasonal_fourier"})
    .hvplot.line("ds", ["y", "trend", "seasonal_fourier"])
)

naive = (
    seasonal_decomposition(df_complex, freq=365)
    .rename({"seasonal": "seasonal_naive"})
    .hvplot.line("ds", ["y", "trend", "seasonal_naive"])
)

naive * fourier

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [ds]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [ds]   (value)